In [2]:
import sys
import os
import fnmatch
import moviepy.editor as mp
import matplotlib.pyplot as plt
# import cv2
# from ffpyplayer.player import MediaPlayer
import scipy.io.wavfile
import numpy as np
from librosa import onset

%matplotlib qt

In [3]:

TEST_FILE_DIR = 'test-clips/greta-van-fleet'
VIDEO_TYPES = ['*.mp4']
AUDIO_TYPE = '.wav'
TMP_DIR = 'tmp'
if not os.path.exists(TMP_DIR):
    os.mkdir(TMP_DIR)

In [4]:
videos = []
audio_paths = []
for fn in os.listdir(TEST_FILE_DIR):
    if any([fnmatch.fnmatch(fn, t) for t in VIDEO_TYPES]):
        print(f'loading {fn}')

        full_path = os.path.join(TEST_FILE_DIR, fn)
        clip = mp.VideoFileClip(full_path)
        videos.append(clip)

        # Save to temp audio file
        audio_fn = os.path.splitext(fn)[0] + AUDIO_TYPE
        audio_path = os.path.join(TMP_DIR, audio_fn)
        if os.path.exists(audio_path):
            print('Audio File already saved')
        else:
            print('Saving to temp path')
            clip.audio.write_audiofile(audio_path)
        audio_paths.append(audio_path)

loading yt5s.com-Highway Tune-Greta Van Fleet-Red Rocks.mp4
Audio File already saved
loading yt5s.com-Greta Van Fleet - Highway Tune - 09-23-2019 Red Rocks.mp4
Audio File already saved
loading yt5s.com-Greta van Fleet at Red Rocks - Highway Tune.mp4
Audio File already saved


In [17]:
audio_signals = []
for audio_path in audio_paths:
    rate, data = scipy.io.wavfile.read(audio_path)
    ch1 = np.array([d[0] for d in data])
    ch2 = np.array([d[1] for d in data])
    
    ch1_float = ch1.astype(np.float32)
    if ch1.dtype == 'int16':
        ch1_float = ch1_float / (2 ** 15)
        
    
    audio_signals.append({
        'data': ch1,
        'data_float': ch1_float,
        'rate': rate,
        'path': audio_path
    })

In [18]:
audio_signals[0]['data_float']

array([0.        , 0.        , 0.        , ..., 0.3244629 , 0.31063843,
       0.28622437], dtype=float32)

In [22]:
onset_strengths = []
for a in audio_signals:
    on_str = onset.onset_strength(a['data_float'], sr=a['rate'])
    onset_strengths.append(on_str)

In [24]:
plt.figure()
for i, o in enumerate(onset_strengths):
    plt.subplot(len(onset_strengths), 1, i+1)
    plt.plot(o)
plt.show()